## POS-tagger и NER

#### Задание 1. Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы сделать выводы


In [1]:
!pip install corus

In [2]:
!pip install pyconll

In [3]:
# Импорт библиотек

import corus
import pandas as pd

import pyconll

import nltk
# nltk.download()
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Загрузка данных

train_data = pyconll.load_from_file('./ru_syntagrus-ud-train-a.conllu')
test_data = pyconll.load_from_file('./ru_syntagrus-ud-dev.conllu')

In [5]:
for sent in test_data[3:5]:
    for token in sent:
        print(token.form, token.upos)
    print()

Таким DET
образом NOUN
, PUNCT
некоторые DET
инструкции NOUN
должны ADJ
выполняться VERB
строго ADV
после ADP
завершения NOUN
работы NOUN
инструкций NOUN
, PUNCT
от ADP
которых PRON
они PRON
зависят VERB
. PUNCT

Независимые ADJ
инструкции NOUN
или CCONJ
инструкции NOUN
, PUNCT
ставшие VERB
независимыми ADJ
из-за ADP
завершения NOUN
работы NOUN
инструкций NOUN
, PUNCT
от ADP
которых PRON
они PRON
зависят VERB
, PUNCT
могут VERB
выполняться VERB
в ADP
произвольном ADJ
порядке NOUN
, PUNCT
параллельно ADV
или CCONJ
одновременно ADV
, PUNCT
если SCONJ
это PRON
позволяют VERB
используемые VERB
процессор NOUN
и CCONJ
операционная ADJ
система NOUN
. PUNCT



In [6]:
fdata_train = []
for sent in train_data:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in test_data:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in test_data:
    fdata_sent_test.append([token.form for token in sent])

In [7]:
results = []

In [8]:
# Default tagger

default_tagger = DefaultTagger('NOUN')

# display(default_tagger.tag(fdata_sent_test[100]), default_tagger.evaluate(fdata_test))
default_tagger.evaluate(fdata_test)

results.append(('default_tagger', default_tagger.evaluate(fdata_test)))

In [9]:
# Unigram Tagger

unigram_tagger = UnigramTagger(fdata_train)
# display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))
unigram_tagger.evaluate(fdata_test)

results.append(('unigram_tagger', unigram_tagger.evaluate(fdata_test)))

In [10]:
# Bigram Tagger

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
# display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))
bigram_tagger.evaluate(fdata_test)

results.append(('bigram_tagger', bigram_tagger.evaluate(fdata_test)))

In [11]:
# Trigram Tagger

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
# display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))
trigram_tagger.evaluate(fdata_test)

results.append(('trigram_tagger', trigram_tagger.evaluate(fdata_test)))

In [12]:
# Комбинация тэггеров

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

backoff = DefaultTagger('NOUN')

tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

results.append(('comb_tagger', tag.evaluate(fdata_test)))

In [13]:
fdata_train[0]

[('Анкета', 'NOUN'), ('.', 'PUNCT')]

In [14]:
fdata_test[:1]

[[('Алгоритм', 'NOUN'),
  (',', 'PUNCT'),
  ('от', 'ADP'),
  ('имени', 'NOUN'),
  ('учёного', 'NOUN'),
  ('аль', 'PART'),
  ('-', 'PUNCT'),
  ('Хорезми', 'PROPN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('точный', 'ADJ'),
  ('набор', 'NOUN'),
  ('инструкций', 'NOUN'),
  (',', 'PUNCT'),
  ('описывающих', 'VERB'),
  ('порядок', 'NOUN'),
  ('действий', 'NOUN'),
  ('исполнителя', 'NOUN'),
  ('для', 'ADP'),
  ('достижения', 'NOUN'),
  ('результата', 'NOUN'),
  ('решения', 'NOUN'),
  ('задачи', 'NOUN'),
  ('за', 'ADP'),
  ('конечное', 'ADJ'),
  ('время', 'NOUN'),
  ('.', 'PUNCT')]]

In [15]:
train_tok = []
train_label = []
for sent in fdata_train:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [16]:
train_tok[0:5], len(train_tok)

(['Анкета', '.', 'Начальник', 'областного', 'управления'], 426182)

In [17]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_label[0:5], train_enc_labels[0:5], len(train_enc_labels)

(['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN'],
 array([ 7, 13,  7,  0,  7], dtype=int64),
 426182)

In [18]:
test_tok[0:5]

['Алгоритм', ',', 'от', 'имени', 'учёного']

In [19]:
test_enc_labels = le.transform(test_label)
test_label[0:5], test_enc_labels[0:5], len(test_enc_labels)

(['NOUN', 'PUNCT', 'ADP', 'NOUN', 'NOUN'], array([ 7, 13,  1,  7,  7]), 153590)

In [20]:
# Hashing Vectorizer

hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=70)

X_train = hvectorizer.fit_transform(train_tok)

X_test = hvectorizer.transform(test_tok)

lr = LogisticRegression(random_state=42, max_iter=15)

lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

results.append(('hashing_vectorizer', accuracy_score(test_enc_labels, pred)))

In [21]:
X_train.shape, X_train[0]

((426182, 70),
 <1x70 sparse matrix of type '<class 'numpy.float64'>'
 	with 14 stored elements in Compressed Sparse Row format>)

In [22]:
# Count Vectorizer

cvectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char')

X_train = cvectorizer.fit_transform(train_tok)

X_test = cvectorizer.transform(test_tok)

lr = LogisticRegression(random_state=0, max_iter=10)

lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

results.append(('count_vectorizer', accuracy_score(test_enc_labels, pred)))

In [23]:
X_train.shape, X_train[0]

((426182, 11987),
 <1x11987 sparse matrix of type '<class 'numpy.int64'>'
 	with 14 stored elements in Compressed Sparse Row format>)

In [24]:
print( X_train[0])

  (0, 2792)	2
  (0, 3084)	1
  (0, 3094)	1
  (0, 4526)	1
  (0, 4925)	1
  (0, 4927)	1
  (0, 6171)	1
  (0, 6236)	1
  (0, 6248)	1
  (0, 7175)	1
  (0, 7341)	1
  (0, 7345)	1
  (0, 9288)	1
  (0, 9306)	1


In [25]:
# Tfidf Vectorizer

tvectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char')

X_train = tvectorizer.fit_transform(train_tok)

X_test = tvectorizer.transform(test_tok)

lr = LogisticRegression(random_state=0, max_iter=10)

lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

results.append(('tfidf_vectorizer', accuracy_score(test_enc_labels, pred)))

In [26]:
# Общие результаты

res_df = pd.DataFrame(results, columns=['approach', 'accuracy'])

res_df.sort_values(by='accuracy', ascending=False)

,approach,accuracy
4,comb_tagger,0.878775
2,bigram_tagger,0.829279
3,trigram_tagger,0.829143
1,unigram_tagger,0.823732
6,count_vectorizer,0.747327
7,tfidf_vectorizer,0.667986
5,hashing_vectorizer,0.623406
0,default_tagger,0.235940


#### Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
- проверить NER из nltk/spacy/deeppavlov
- написать свой нер попробовать разные подходы
-- передаём в сетку токен и его соседей
-- передаём в сетку только токен
- сделать выводы по вашим экспериментам какой из подходов успешнее справляется


In [36]:
# Загрузка данных

from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)

In [37]:
next(records)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 171: character maps to <undefined>

In [112]:
text = 'Д.Медведев снял с должности замсекретаря Совбеза РФ Ю.Балуевского\r\n\r\nПрезидент России Дмитрий Медведев освободил Юрия Балуевского от должности замсекретаря Совета безопасности России. Соответствующий указ опубликован на сайте государственной системы правовой информации.\r\n\r\nУказ глава государства подписал 9 января. Именно в этот день Ю.Балуевскому исполнилось 65 лет.\r\n\r\nСогласно российскому законодательству, предельный возраст для нахождения на государственной службе составляет 60 лет, однако он может быть увеличен до 65 лет. Дальнейшее продление нахождения на госслужбе не допускается.\r\n\r\nЮ.Балуевский родился 9 января 1947г. в городе Трускавец Львовской области Украины. В 1970г. окончил Ленинградское общевойсковое училище, в 1980г. - Военную академию имени Фрунзе, в 1990г. - Военную академию генштаба Вооруженных сил СССР.\r\n\r\n19 июля 2004г. указом президента Ю.Балуевский был назначен начальником Генерального штаба Вооруженных сил РФ - первым заместителем Министра обороны.\r\n\r\n3 июня 2008г. освобожден от должности начальника Генерального штаба и назначен заместителем секретаря Совета безопасности Российской Федерации. Ему на смену пришел генерал армии Н.Макаров.\r\n\r\nС сентября 2004г. по июнь 2008г. одновременно был членом Совета безопасности Российской Федерации.\r\n\r\nИмеет награды: орден "За заслуги перед Отечеством" II степени - за заслуги перед государством и значительный вклад в дело защиты Отечества; орден "За заслуги перед Отечеством" III степени - за большой вклад в укрепление обороноспособности РФ и многолетнюю добросовестную службу; орден "За заслуги перед Отечеством" IV степени; орден "За военные заслуги"; орден Почета - за большой вклад в обеспечение национальной безопасности РФ и многолетнюю добросовестную работу; орден "За службу Родине в Вооруженных Силах СССР" III степени.\r\n\r\nЮрий Балуевский женат, имеет двух детей.\r\n\r\n'

In [113]:
import re

text = re.sub(r'\r\n\r\n', ' ', text)
text

'Д.Медведев снял с должности замсекретаря Совбеза РФ Ю.Балуевского Президент России Дмитрий Медведев освободил Юрия Балуевского от должности замсекретаря Совета безопасности России. Соответствующий указ опубликован на сайте государственной системы правовой информации. Указ глава государства подписал 9 января. Именно в этот день Ю.Балуевскому исполнилось 65 лет. Согласно российскому законодательству, предельный возраст для нахождения на государственной службе составляет 60 лет, однако он может быть увеличен до 65 лет. Дальнейшее продление нахождения на госслужбе не допускается. Ю.Балуевский родился 9 января 1947г. в городе Трускавец Львовской области Украины. В 1970г. окончил Ленинградское общевойсковое училище, в 1980г. - Военную академию имени Фрунзе, в 1990г. - Военную академию генштаба Вооруженных сил СССР. 19 июля 2004г. указом президента Ю.Балуевский был назначен начальником Генерального штаба Вооруженных сил РФ - первым заместителем Министра обороны. 3 июня 2008г. освобожден от д

In [114]:
# nltk

{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('III', 'ORGANIZATION'),
 ('Балуевский', 'ORGANIZATION'),
 ('Военную', 'PERSON'),
 ('Ленинградское', 'PERSON'),
 ('Министра', 'ORGANIZATION'),
 ('Почета', 'PERSON'),
 ('Родине', 'PERSON'),
 ('России Дмитрий Медведев', 'PERSON'),
 ('Совбеза РФ', 'PERSON'),
 ('Совета', 'PERSON'),
 ('Трускавец Львовской', 'PERSON')}

In [115]:
# spacy

import spacy
import ru_core_news_sm
from spacy import displacy

nlp = ru_core_news_sm.load()

docs = nlp(text)
displacy.render(docs, jupyter=True, style='ent')

Даже по одному короткому отрывку видно, насколько лучше с задачей NER справляется Spacy.